# 🗺️ Geospatial Data Visualizations with Python

## Project Overview

This notebook contains a collection of map-based data visualizations created using Python. It showcases the use of mapping libraries to display spatial patterns and insights from various datasets.

These visualizations are part of a broader data visualization project (#30daymapchallenge) which was published in my LinkedIn account.

## 📦 Importing Libraries and Setup
Set up the environment and import all necessary packages.

In [ ]:
# ===========================
# 📦 Installation (Colab only)
# ===========================
# Run these only if you're on Google Colab
!pip install -q keplergl osmnx networkx rtree h3 contextily cairosvg

# ===========================
# 📚 Import Libraries
# ===========================

# Data Processing
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import json
import requests
import warnings
warnings.filterwarnings('ignore')

# Spatial Tools
import shapely
from shapely.geometry import Point, LineString, Polygon, mapping
from shapely.ops import transform
import rtree
import h3
import pyproj

# Visualization
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import contextily as ctx
import ipywidgets as widgets

# Interactive Mapping
import folium
from folium.plugins import HeatMap, MarkerCluster
from branca.colormap import linear
from keplergl import KeplerGl

# Network & Routing
import networkx as nx
import osmnx as ox
ox.config(log_console=True, use_cache=True)

# PIL and miscellaneous
from PIL import Image
import base64
import tempfile
import urllib

# Google Colab settings (optional)
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

# Inline plotting for Jupyter
%matplotlib inline


## **Day 2: Lines**

In [ ]:
# # download and project Manhattan's street network
G = ox.graph_from_place('Manhattan, New York, USA', network_type='drive')
G = ox.project_graph(G)
fig, ax = ox.plot_graph(G, bgcolor='k', node_size=0.5, node_color='Gold', node_edgecolor='none', node_zorder=2,
edge_color='w', edge_linewidth=0.3, edge_alpha=1)

- Calculate and visualize edge centrality

In [ ]:
# edge closeness centrality: convert graph to a line graph so edges become nodes and vice versa
edge_centrality = nx.closeness_centrality(nx.line_graph(G))

In [ ]:
# List of edge values for the original graph
ev = [edge_centrality[edge + (0,)] for edge in G.edges()]
# Color scale converted to list of colors for graph edges
norm = mcolors.Normalize(vmin=min(ev) * 0.5, vmax=max(ev))
cmap = cm.ScalarMappable(norm=norm, cmap=cm.inferno)
ec = [cmap.to_rgba(cl) for cl in ev]
# Color the edges in the original graph with closeness centralities in the line graph
fig, ax = ox.plot_graph(G, bgcolor='k', node_size=0, node_color='w', node_edgecolor='gray', node_zorder=2,
edge_color=ec, edge_linewidth=0.7, edge_alpha=1)
# Turn off axis
ax.set_axis_off()
# Show the plot
plt.show()

## **Day 7: Navigation**

In [ ]:
# Define the location (Amsterdam)
place_name = "Amsterdam, Netherlands"
# Fetch the bike network graph for the specified location
G = ox.graph_from_place(place_name, network_type='bike')

In [ ]:
# Set central location to the Van Gogh Museum coordinates
central_location = (52.3581, 4.8812)  # Latitude, Longitude
# Define the bike distance in 15 minutes
dist = 3500  # This should be in meters, approximately 15 minutes of biking
# Calculate the reachable locations within the given time
reachable_graph = ox.graph_from_point(
central_location, dist=dist, network_type="bike", dist_type='network'
)
# Create a GeoDataFrame from the reachable graph
reachable_edge = ox.graph_to_gdfs(reachable_graph, nodes=False, edges=True)

In [ ]:
# Create a GeoDataFrame from the reachable graph
amsterdam_edge = ox.graph_to_gdfs(G, nodes=False, edges=True)

In [ ]:
# Load the base map of Amsterdam
amsterdam_map = folium.Map(location=central_location, zoom_start=12, tiles="CartoDB Dark_Matter")
# Define a style function to make the lines thinner
def style_function(feature):
return {
'color': 'orange',  # Line color
'weight': 1,      # Line weight
}
# Define a base function to make the lines thinner
def base_function(feature):
return {
'color': 'white',  # Line color
'weight': 0.5,      # Line weight
}
# Plot the GeoDataFrame on the map with the updated style
gdf_layer = folium.GeoJson(amsterdam_edge, style_function=base_function).add_to(amsterdam_map)
# Plot the GeoDataFrame on the map with the updated style
gdf_layer = folium.GeoJson(reachable_edge, style_function=style_function).add_to(amsterdam_map)
# Create a custom icon using your custom picture
icon = folium.CustomIcon(
icon_image='/content/image-from-rawpixel-id-6739881-png.png', #Upload a PNG for Van Gogh's icon image in the map
icon_size=(40, 40),  # Adjust the size
)
# Add the custom icon to the map at a specific location
marker_location = Point(4.8812, 52.3581)  # Longitude, Latitude
marker = folium.Marker(
location=[marker_location.y, marker_location.x],
popup="Van Gogh Museum",
icon=icon,
)
marker.add_to(amsterdam_map)
# Display the map
#amsterdam_map.save("amsterdam_map.html")
amsterdam_map

## **Day 8: Africa**

In [ ]:
urbtrend= pd.read_excel("https://datacatalogfiles.worldbank.org/ddh-published/0060310/DR0084587/WB-SMSUA_DLR_UrbTrends_v01.xlsx")

In [ ]:
urbcenter= pd.read_excel("https://datacatalogfiles.worldbank.org/ddh-published/0060310/DR0084441/WB-SMSUA_DLR_%20UrbanCenters_v08.xlsx")

- Data Source: https://datacatalogfiles.worldbank.org/ddh-published/0060310/DR0084581/UrbanClusters.zip

In [ ]:
gdf= gpd.read_file("/content/UC_v8_WB.shp")

In [ ]:
# Create a base map centered at Palo Alto
africa = folium.Map(location=[8.7832, 34.5085], zoom_start=4)
# Convert the data into a list of (latitude, longitude, value) tuples
heatmap_data = [
(row['geometry'], row['Population']) for index, row in gdf.iterrows()
]
# Determine the range of values for the colorbar
min_value = min(heatmap_data, key=lambda x: x[1])[1]
max_value = max(heatmap_data, key=lambda x: x[1])[1]
# Create a heatmap layer with default colormap
heatmap_layer = HeatMap(heatmap_data, radius=20, blur=15)
heatmap_layer.add_to(pcmap)
heatmap_layer.layer_name = 'Heatmap of Frequent Events'
# Add colorbar legend
colormap_caption = 'Events per Day'
colormap = folium.LinearColormap(colors=['steelblue', 'lime', 'yellow', 'orange', 'red'], vmin=min_value, vmax=max_value)
colormap.caption = colormap_caption
pcmap.add_child(colormap)
# Create a feature group for the markers
marker_group = folium.FeatureGroup(name='Postal Code Markers')
# Iterate over postal codes and add markers with pop-up information
for index, row in pc_total_events.iterrows():
postal_code = row['postal_code']
events = row['Event_p_Day']
latitude = row['latitude']
longitude = row['longitude']
color = colormap(events)
# Create the marker
marker = folium.CircleMarker(
[latitude, longitude], radius=6, color=color, fill=True, fill_color=color,opacity=0.001, fill_opacity=0.001,
popup=f'Postal Code: {postal_code}, Avg Events per Day: {events}').add_to(marker_group)
# Add the marker group to the map, but keep it initially hidden
marker_group.add_to(pcmap)
marker_group.layer_name = 'Postal Code Popup'
# Add a layer control to toggle visibility of the layers
layer_control = folium.LayerControl(collapsed=False).add_to(pcmap)
# Show map
pcmap

## **Day 9: Hexagons**

In [ ]:
#Functions
def plot_scatter(df, metric_col, x='lng', y='lat', marker='.', alpha=1, figsize=(16,12), colormap='viridis'):
df.plot.scatter(x=x, y=y, c=metric_col, title=metric_col
, edgecolors='none', colormap=colormap, marker=marker, alpha=alpha, figsize=figsize);
plt.xticks([], []); plt.yticks([], [])
def aperture_downsampling(df, hex_col, metric_col, coarse_aperture_size):
df_coarse = df.copy()
coarse_hex_col = 'hex{}'.format(coarse_aperture_size)
df_coarse[coarse_hex_col] = df_coarse[hex_col].apply(lambda x: h3.h3_to_parent(x,coarse_aperture_size))
dfc = df_coarse.groupby([coarse_hex_col])[[metric_col,]].mean().reset_index()
dfc['lat'] = dfc[coarse_hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
dfc['lng'] = dfc[coarse_hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
return dfc
def kring_smoothing(df, hex_col, metric_col, k):
dfk = df[[hex_col]]
dfk.index = dfk[hex_col]
dfs =  (dfk[hex_col]
.apply(lambda x: pd.Series(list(h3.k_ring(x,k)))).stack()
.to_frame('hexk').reset_index(1, drop=True).reset_index()
.merge(df[[hex_col,metric_col]]).fillna(0)
.groupby(['hexk'])[[metric_col]].sum().divide((1 + 3 * k * (k + 1)))
.reset_index()
.rename(index=str, columns={"hexk": hex_col}))
dfs['lat'] = dfs[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
dfs['lng'] = dfs[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
return dfs
def weighted_kring_smoothing(df, hex_col, metric_col, coef):
# normalize the coef
a = []
for k, coe in enumerate(coef):
if k == 0:
a.append(coe)
else:
a.append(k * 6 * coe)
coef = [c / sum(a) for c in coef]
# weighted smoothing
df_agg = df[[hex_col]]
df_agg['hexk'] = df_agg[hex_col]
df_agg.set_index(hex_col,inplace=True)
temp2 = [df_agg['hexk'].reset_index()]
temp2[-1]['k'] = 0
K=len(coef)-1
for k in range(1,K+1):
temp2.append((df_agg['hexk']
.apply(lambda x: pd.Series(list(h3.hex_ring(x,k)))).stack()
.to_frame('hexk').reset_index(1, drop=True).reset_index()
))
temp2[-1]['k'] = k
df_all = pd.concat(temp2).merge(df)
df_all[metric_col] = df_all[metric_col]*df_all.k.apply(lambda x:coef[x])
dfs = df_all.groupby('hexk')[[metric_col]].sum().reset_index().rename(index=str, columns={"hexk": hex_col})
dfs['lat'] = dfs[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
dfs['lng'] = dfs[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
return dfs

In [ ]:
crime= pd.read_csv('https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?date=20231109&accessType=DOWNLOAD')

In [ ]:
dfcrime= crime[["Latitude",	"Longitude"]]

In [ ]:
dfcrime= dfcrime.dropna()

In [ ]:
dfcrime= dfcrime[dfcrime["Latitude"] > 41]

In [ ]:
# Visualize the crash points
dfcrime.plot(x='Latitude',y='Longitude',style='.',alpha=1,figsize=(12,12))
plt.title('Sample Points: Crime Locations')

In [ ]:
def plot_scatter(df, metric_col, x='Latitude', y='Longitude', marker='.', alpha=1, figsize=(16,12), colormap='coolwarm'):
df.plot.scatter(x=x, y=y, c=metric_col, title=metric_col
, edgecolors='none', colormap=colormap, marker=marker, alpha=alpha, figsize=figsize);
plt.xticks([], []); plt.yticks([], [])

In [ ]:
APERTURE_SIZE = 9
hex_col = 'hex'+str(APERTURE_SIZE)
# find hexs containing the points
dfcrime[hex_col] = dfcrime.apply(lambda x: h3.geo_to_h3(x.Latitude,x.Longitude,APERTURE_SIZE),1)
# aggregate the points
dfcrimeg = dfcrime.groupby(hex_col).size().to_frame('cnt').reset_index()
#find center of hex for visualization
dfcrimeg['Latitude'] = dfcrimeg[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
dfcrimeg['Longitude'] = dfcrimeg[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
# pltot the hexs
plot_scatter(dfcrimeg, metric_col='cnt', marker='o',figsize=(17,15))
plt.title('hex-grid: Traffic Crashes')

In [ ]:
def plot_smooth_scatter(df, metric_col, x='lat', y='lng', marker='.', alpha=1, figsize=(16,12), colormap='coolwarm'):
df.plot.scatter(x=x, y=y, c=metric_col, title=metric_col
, edgecolors='none', colormap=colormap, marker=marker, alpha=alpha, figsize=figsize);
plt.xticks([], []); plt.yticks([], [])

In [ ]:
#kring_smoothing
k = 2
dfcrimes= kring_smoothing(dfcrimeg, hex_col, metric_col='cnt', k=k)
print('sum sanity check:', dfcrimes['cnt'].sum() / dfcrimes['cnt'].sum())
plot_smooth_scatter(dfcrimes, metric_col='cnt', marker='o')
plt.title('Traffic Crashes: 2-ring average')

## **Day 10: North America**

- Data Source:  https://www.kaggle.com/datasets/debdutta/cost-of-living-index-by-country

In [ ]:
df= pd.read_csv("/content/Cost_of_living_index.csv")

In [ ]:
df.head()

In [ ]:
# Filtering US and Canada cities
filtered_df = df[df['City'].str.contains('United States|Canada')]

In [ ]:
filtered_df.head()

In [ ]:
# Split the 'City' column to extract only the city names
filtered_df['City'] = filtered_df['City'].str.split(',').str[0]
# Now, 'City' contains only the city names"

In [ ]:
filtered_df.head()

In [ ]:
# Read the CSV files containing US and Canadian cities' coordinates
us_cities_df = pd.read_csv('/content/uscities.csv')
canadian_cities_df = pd.read_csv('/content/canadacities.csv')
# Concatenate the two DataFrames to create a single DataFrame with all city coordinates
all_cities_df = pd.concat([us_cities_df, canadian_cities_df])
# Merge the combined coordinates DataFrame with filtered_df using the 'city_ascii' column as the key
merged_df = filtered_df.merge(all_cities_df, left_on='City', right_on='city_ascii', how='left')

In [ ]:
# Drop duplicates in the 'merged_df' based on the 'City' column
merged_df = merged_df.drop_duplicates(subset=['City'])

In [ ]:
print(merged_df.columns)

In [ ]:
map_df = merged_df[['City', 'Cost of Living Plus Rent Index', 'lat', 'lng']]

In [ ]:
map_df.head()

In [ ]:
# Create a Kepler.gl map
map_1 = KeplerGl(height=600, data={"data": map_df}, config={})
map_1

## **Day 15: OSM**

https://geoffboeing.com/2017/08/isochrone-maps-osmnx-python/

https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.graph.graph_from_point

https://github.com/gboeing/osmnx-examples/blob/main/notebooks/13-isolines-isochrones.ipynb

https://wiki.openstreetmap.org/wiki/Map_features

In [ ]:
# configure the place, network type, trip times, and travel speed
place = 'Paris, France'
network_type = "walk"
trip_times = [5, 10, 15, 20, 25]  # in minutes
travel_speed = 4.5  # walking speed in km/hour

In [ ]:
# download the street network
G = ox.graph_from_place(place, network_type=network_type)

In [ ]:
Y= 48.8606
X = 2.3376
center_node = ox.distance.nearest_nodes(G, X, Y)
G = ox.project_graph(G)

In [ ]:
# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for _, _, _, data in G.edges(data=True, keys=True):
data["time"] = data["length"] / meters_per_minute

In [ ]:
# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap="plasma", start=0)#, return_hex=True)

In [ ]:
# color the nodes according to isochrone then plot the street network
node_colors = {}
for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
for node in subgraph.nodes():
node_colors[node] = color
nc = [node_colors[node] if node in node_colors else "none" for node in G.nodes()]
ns = [15 if node in node_colors else 0 for node in G.nodes()]
fig, ax = ox.plot_graph(
G,
node_color=nc,
node_size=ns,
node_alpha=0.8,
edge_linewidth=0.2,
edge_color="#999999",
)

In [ ]:
# make the isochrone polygons
isochrone_polys = []
for trip_time in sorted(trip_times, reverse=True):
subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
isochrone_polys.append(bounding_poly)
gdf = gpd.GeoDataFrame(geometry=isochrone_polys)

In [ ]:
# plot the network then add isochrones as colored polygon patches
fig, ax = ox.plot_graph(
G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)
gdf.plot(ax=ax, color=iso_colors, ec="none", alpha=0.6, zorder=-1)
plt.show()

In [ ]:
def make_iso_polys(G, edge_buff=25, node_buff=50, infill=False):
isochrone_polys = []
for trip_time in sorted(trip_times, reverse=True):
subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
nodes_gdf = nodes_gdf.set_index("id")
edge_lines = []
for n_fr, n_to in subgraph.edges():
f = nodes_gdf.loc[n_fr].geometry
t = nodes_gdf.loc[n_to].geometry
edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
edge_lines.append(edge_lookup)
n = nodes_gdf.buffer(node_buff).geometry
e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
all_gs = list(n) + list(e)
new_iso = gpd.GeoSeries(all_gs).unary_union
# try to fill in surrounded areas so shapes will appear solid and
# blocks without white space inside them
if infill:
new_iso = Polygon(new_iso.exterior)
isochrone_polys.append(new_iso)
return isochrone_polys

In [ ]:
# make the isochrone polygons
isochrone_polys = make_iso_polys(G, edge_buff=25, node_buff=0, infill=True)
gdf = gpd.GeoDataFrame(geometry=isochrone_polys)
# plot the network then add isochrones as colored polygon patches
fig, ax = ox.plot_graph(
G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0)
gdf.plot(ax=ax, color=iso_colors, ec="none", alpha=0.6, zorder=-1)
plt.show()

## **Day 16: Oceania**

In [ ]:
# Define the location (Amsterdam)
place_name = "Auckland, New Zealand"
# Fetch the bike network graph for the specified location
#G = ox.graph_from_place(place_name, network_type='walk')#, simplify=False)

In [ ]:
gdf= gpd.read_file('https://data-atgis.opendata.arcgis.com/datasets/ATgis::train-station.geojson?outSR=%7B%22latestWkid%22%3A2193%2C%22wkid%22%3A2193%7D')

In [ ]:
#gdf['STOPLAT']= gdf['STOPLAT'].astype('float')
#gdf['STOPLON']= gdf['STOPLON'].astype('float')

In [ ]:
gdf= gdf[gdf['STOPLAT']> -37.25]

In [ ]:
# Initialize an empty GeoDataFrame to store reachable edges
reachable_edges_gdf = gpd.GeoDataFrame()
# Iterate over each station
for index, station in gdf.iterrows():
location = (station['geometry'].y, station['geometry'].x)  # Note the order of lat, lon
# Initialize an empty GeoDataFrame for each station
station_reachable_edges_gdf = gpd.GeoDataFrame()
isochrone_walk_distance = 1125
# Calculate the reachable locations within the given time
reachable_graph = ox.graph_from_point(location, dist=isochrone_walk_distance, network_type="walk", dist_type='network')
# Create a GeoDataFrame from the reachable graph
reachable_edge = ox.graph_to_gdfs(reachable_graph, nodes=False, edges=True)
# Add the reachable edges to the station-specific GeoDataFrame
station_reachable_edges_gdf = pd.concat([station_reachable_edges_gdf, reachable_edge], ignore_index=True)
# Add the station-specific GeoDataFrame to the overall GeoDataFrame
reachable_edges_gdf = pd.concat([reachable_edges_gdf, station_reachable_edges_gdf], ignore_index=True)

In [ ]:
train = gpd.read_file('https://data-atgis.opendata.arcgis.com/datasets/ATgis::train-route.geojson?where=1=1&outSR=%7B%22latestWkid%22%3A2193%2C%22wkid%22%3A2193%7D')

In [ ]:
boundry= gpd.read_file('http://data-aucklandcouncil.opendata.arcgis.com/datasets/89052c6dc2634cf987539f3f188ffd76_0.geojson?outSR={%22latestWkid%22:3857,%22wkid%22:102100}')

- http://data-aucklandcouncil.opendata.arcgis.com/datasets/5d35cbf5ff1c4baa85023c4b4c7de24f_0.geojson?outSR={%22latestWkid%22:3857,%22wkid%22:102100}  https://catalogue.data.govt.nz/dataset/auckland-council-boundary-area  https://catalogue.data.govt.nz/dataset/auckland-council-ward  https://catalogue.data.govt.nz/dataset/auckland-council-subdivision  http://data-aucklandcouncil.opendata.arcgis.com/datasets/89052c6dc2634cf987539f3f188ffd76_0.geojson?outSR={%22latestWkid%22:3857,%22wkid%22:102100}

In [ ]:
boundry

In [ ]:
boundry.plot()

In [ ]:
# Define a custom HTML icon with Font Awesome
icon_html = '<i class="fa-solid fa-train-subway fa-xl" style="color: red;"></i>'
# Coordinates for Paris, France
au_coords = [-36.848461, 174.763336]
# Plotting
m = folium.Map(location=au_coords, zoom_start=14, tiles="CartoDB Dark_Matter")
# Define a style function to make the lines thinner
def route_function(feature):
return {
'color': 'orange',  # Line color
'weight': 1,      # Line weight
}
# Define a style function to make the lines thinner
def style_function(feature):
return {
'color': 'green',  # Line color
'weight': 0.5,      # Line weight
}
# Plot the GeoDataFrame on the map with the updated style
gdf_layer = folium.GeoJson(train, style_function=route_function).add_to(m)
# Plot the GeoDataFrame on the map with the updated style
gdf_layer = folium.GeoJson(reachable_edges_gdf, style_function=style_function).add_to(m)
# Add stations to the map with custom icons
for _, station in gdf.iterrows():
latitude, longitude = station['geometry'].y, station['geometry'].x
marker = folium.Marker(
location=[latitude, longitude],
icon=folium.DivIcon(html=icon_html),
)
marker.add_to(m)
# Save the map as an HTML file
html_file_path = "custom_map2.html"
#m.save(html_file_path)
m

## **Day 17: Flow**

- COMMUNITY AREAS

https://data.cityofchicago.org/api/geospatial/cauq-8yn6?method=export&format=GeoJSON

https://data.cityofchicago.org/api/views/igwz-8jzy/rows.csv?accessType=DOWNLOAD

In [ ]:
gdf= gpd.read_file('https://data.cityofchicago.org/api/geospatial/cauq-8yn6?method=export&format=GeoJSON')

In [ ]:
df= pd.read_csv('/content/map.csv')

In [ ]:
df.head()

In [ ]:
# Create a Kepler.gl map
map_1 = KeplerGl(height=600)
# Add the GeoDataFrame as a layer
map_1.add_data(data=df, name='Acc')
map_1.add_data(data=gdf, name='Bound')
# Show the map
map_1

## **Day 28: Is this a chart or a map?**

In [ ]:
query= 'Brisbane, Australia'
G = ox.graph_from_place(query, network_type='drive')

In [ ]:
Gu= ox.utils_graph.get_undirected(G)

In [ ]:
Gu= ox.bearing.add_edge_bearings(Gu, precision=None)

In [ ]:
col= ox.plot.get_edge_colors_by_attr(Gu, 'length', num_bins=4, cmap='winter', start=0, stop=100000, na_color='none', equal_size=False)

In [ ]:
color= ox.plot.get_colors(4, cmap='winter', start=0.0, stop=100.0, alpha=1.0, return_hex=False)

In [ ]:
ox.plot.plot_orientation(Gu, num_bins=36, min_length=0, weight='length', ax=None,
figsize=(10, 10), area=True, color=col, edgecolor='k',
linewidth=0.5, alpha=0.7)

In [ ]:
# define the study sites as label : query
places = {'Darwin'       : 'City of Darwin, Australia',
'Townsville'       : 'Townsville, Australia',
'Australian Capital Territory'       : 'Australian Capital Territory, Australia',
'Hobart'       : 'Hobart, Australia'
}

- Get the street networks and their edge bearings

In [ ]:
def reverse_bearing(x):
return x + 180 if x < 180 else x - 180

In [ ]:
bearings = {}
for place in sorted(places.keys()):
print(datetime.datetime.now(), place)
# get the graph
query = places[place]
G = ox.graph_from_place(query, network_type='drive')
# calculate edge bearings
Gu = ox.bearing.add_edge_bearings(ox.utils_graph.get_undirected(G))
if weight_by_length:
# weight bearings by length (meters)
city_bearings = []
for u, v, k, d in Gu.edges(keys=True, data=True):
bearing = d.get('bearing', None)  # use get method to avoid KeyError
if bearing is not None:
city_bearings.extend([bearing] * int(d['length']))
b = pd.Series(city_bearings)
bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
else:
# don't weight bearings, just take one value per street segment
b = pd.Series([d.get('bearing', None) for u, v, k, d in Gu.edges(keys=True, data=True)])
bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

- Visualize it

In [ ]:
def count_and_merge(n, bearings):
# make twice as many bins as desired, then merge them in pairs
# prevents bin-edge effects around common values like 0° and 90°
n = 36 * 2
bins = np.arange(n + 1) * 360 / n
count, _ = np.histogram(bearings, bins=bins)
# move the last bin to the front, so eg 0.01° and 359.99° will be binned together
count = np.roll(count, 1)
return count[::2] + count[1::2]

In [ ]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):
bins = np.arange(n + 1) * 360 / n
count = count_and_merge(n, bearings)
_, division = np.histogram(bearings, bins=bins)
frequency = count / count.sum()
division = division[0:-1]
width =  2 * np.pi / n
ax.set_theta_zero_location('N')
ax.set_theta_direction('clockwise')
x = division * np.pi / 180
cmap = plt.get_cmap('winter')
norm = plt.Normalize(0, frequency.max())
colors = cmap(norm(frequency))
bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
color=colors, edgecolor='k', linewidth=0.5, alpha=0.7)
ax.set_ylim(top=frequency.max())
title_font = {'family':'Century Gothic', 'size':24, 'weight':'bold'}
xtick_font = {'family':'Century Gothic', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
ytick_font = {'family':'Century Gothic', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
ax.set_title(title.upper(), y=1.05, fontdict=title_font)
ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
yticklabels[0] = ''
ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
ax.tick_params(axis='x', which='major', pad=-2)

In [ ]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})
# plot each city's polar histogram
for ax, place in zip(axes.flat, sorted(places.keys())):
polar_plot(ax, bearings[place].dropna(), title=place)
# add super title and save full image
suptitle_font = {'family':'Century Gothic', 'fontsize':60, 'fontweight':'normal', 'y':1.07}
fig.suptitle('City Street Network Orientation', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
#fig.savefig('/content/street-orientationnn.png', dpi=300, bbox_inches='tight')
#plt.close()
plt.show()

In [ ]:
# City coordinates (latitude, longitude)
cities = {
'Adelaide': (-34.9285, 138.6007),
'Brisbane': (-27.4698, 153.0251),
'Melbourne': (-37.8136, 144.9631),
'Perth': (-31.9505, 115.8605),
'Sydney': (-33.8688, 151.2093),
'Canberra': (-35.2820, 149.1287),
'Hobart': (-42.8821, 147.3272),
'Townsville': (-19.2576, 146.8183),
'Darwin': (-12.4634, 130.8456)
}
# Path to the images
image_paths = {
'Adelaide': '/content/Adelaide.png',
'Brisbane': '/content/brisbane.png',
'Melbourne': '/content/melbourne.png',
'Perth': '/content/perth.png',
'Sydney': '/content/sydney.png',
'Canberra': '/content/canberra.png',
'Hobart': '/content/hobart.png',
'Townsville': '/content/townswille.png',
'Darwin': '/content/darwin.png'
}
# Create a folium map with CartoDB Dark Matter as the base map
australia_map = folium.Map(location=[-25.2744, 133.7751], zoom_start=4, tiles='cartodb dark_matter')
# Add markers for each city
for city, (lat, lon) in cities.items():
# Create a folium marker for each city
marker = folium.Marker(location=(lat, lon), popup=city)
# Add the city image to the marker icon
icon = folium.CustomIcon(icon_image=image_paths[city], icon_size=(50, 50))
marker.add_child(icon)
# Add the marker to the map
australia_map.add_child(marker)
# Display the map
australia_map

## **Day 30: My Favorite**

In [ ]:
# configure the place, network type, trip times, and travel speed
place = 'Amsterdam, Netherlands'
network_type = "walk"
trip_times = [5, 10, 15]  # in minutes
travel_speed = 4.5  # walking speed in km/hour

In [ ]:
# download the street network
G = ox.graph_from_place(place, network_type=network_type, simplify=False)

In [ ]:
# Download bicycle_rental data
bc= ox.features.features_from_place(place, tags={'amenity':'bicycle_rental'})

In [ ]:
points= bc.reset_index()

In [ ]:
# Filter only the points
#points = bc[bc['geometry'].type == 'Point']

In [ ]:
# Filter only the points
points= points[points['element_type']=='node']

In [ ]:
# Get the nearest node
center_node=[]
for point in points.geometry:
X= point.x
Y= point.y
node = ox.distance.nearest_nodes(G, X, Y)
center_node.append(node)

In [ ]:
# Project graph
G = ox.project_graph(G)

In [ ]:
# Add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = (travel_speed * 1000) / 60  # km per hour to m per minute
for _, _, _, data in G.edges(data=True, keys=True):
data["time"] = data["length"] / meters_per_minute

In [ ]:
# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap="winter", start=0, return_hex=True)

In [ ]:
# color the nodes according to isochrone then plot the street network
node_colors = {}
for n_c in center_node:
for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
subgraph = nx.ego_graph(G, n_c, radius=trip_time, distance="time")
for node in subgraph.nodes():
node_colors[node] = color
nc = [node_colors[node] if node in node_colors else "none" for node in G.nodes()]
ns = [10 if node in node_colors else 0 for node in G.nodes()]
fig, ax = ox.plot_graph(
G,
node_color=nc,
node_size=ns,
node_alpha=0.8,
edge_linewidth=0.2,
edge_color="#999999",
)

In [ ]:
# make the isochrone polygons
isochrone_polys = []
isochrone_colors = []
isochrone_times = []
for n_c in center_node:
for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
subgraph = nx.ego_graph(G, n_c, radius=trip_time, distance="time")
node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
isochrone_polys.append(bounding_poly)
isochrone_colors.append(color)
isochrone_times.append(trip_time)
# create a GeoDataFrame with isochrone polygons and colors
gdf = gpd.GeoDataFrame(geometry=isochrone_polys)
gdf['time'] = isochrone_times
gdf['color'] = isochrone_colors
# sort GeoDataFrame based on time
gdf = gdf.sort_values(by='time', ascending=False)

In [ ]:
# plot the network then add isochrones as colored polygon patches
fig, ax = ox.plot_graph(
G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)
gdf.plot(ax=ax, color=gdf['color'], ec="none", alpha=0.6, zorder=-1)
plt.show()

In [ ]:
def make_iso_polys(G, center_nodes, edge_buff=25, node_buff=50, infill=False):
isochrone_polys = []
isochrone_times = []
isochrone_colors = []
for center_node in center_nodes:
for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
nodes_gdf = nodes_gdf.set_index("id")
edge_lines = []
for n_fr, n_to in subgraph.edges():
f = nodes_gdf.loc[n_fr].geometry
t = nodes_gdf.loc[n_to].geometry
edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
edge_lines.append(edge_lookup)
n = nodes_gdf.buffer(node_buff).geometry
e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
all_gs = list(n) + list(e)
new_iso = gpd.GeoSeries(all_gs).unary_union
# try to fill in surrounded areas so shapes will appear solid and
# blocks without white space inside them
if infill:
new_iso = Polygon(new_iso.exterior)
isochrone_polys.append(new_iso)
isochrone_times.append(trip_time)
isochrone_colors.append(color)
gdf = gpd.GeoDataFrame(geometry=isochrone_polys)
gdf['time'] = isochrone_times
gdf['color'] = isochrone_colors
return gdf

In [ ]:
# make the isochrone polygons
gdf = make_iso_polys(G, center_nodes=center_node, edge_buff=25, node_buff=0, infill=True)

In [ ]:
# sort GeoDataFrame based on time
gdf = gdf.sort_values(by='time', ascending=False)

In [ ]:
# plot the network then add isochrones as colored polygon patches
fig, ax = ox.plot_graph(
G, show=False, close=False, edge_color="#999999", edge_alpha=0.2, node_size=0
)
gdf.plot(ax=ax, color=gdf['color'], ec="none", alpha=0.6, zorder=-1)
plt.show()

In [ ]:
# Create a GeoDataFrame from the city graph
amsterdam = ox.graph_to_gdfs(G, nodes=False, edges=True)

In [ ]:
# Export GeoDataFrame to a Shapefile
output_shapefile_path = 'amsterdam.shp'
amsterdam.to_file(output_shapefile_path, driver='ESRI Shapefile')
# Display the path to the exported Shapefile
print(f"GeoDataFrame exported to: {output_shapefile_path}")

In [ ]:
# Export GeoDataFrame to a Shapefile
output_shapefile_path = 'isochrones.shp'
gdf.to_file(output_shapefile_path, driver='ESRI Shapefile')
# Display the path to the exported Shapefile
print(f"GeoDataFrame exported to: {output_shapefile_path}")

In [ ]:
# Export GeoDataFrame to a Shapefile
output_shapefile_path = 'points.shp'
points.to_file(output_shapefile_path, driver='ESRI Shapefile')
# Display the path to the exported Shapefile
print(f"GeoDataFrame exported to: {output_shapefile_path}")

In [ ]:
gdf.crs = 'EPSG:4326'

In [ ]:
# Plot the GeoDataFrame
ax = gdf.plot(figsize=(20, 20), color=gdf['color'], edgecolor="none", alpha=0.6, zorder=-1)
amsterdam.plot(color='grey', linewidth=0.1, ax=ax)
# Create a colorbar legend
cbar = plt.colorbar(plt.cm.ScalarMappable(cmap='YlOrRd'), ax=ax, orientation='vertical', fraction=0.03, pad=0.02)
cbar.set_label('Walking Minutes')  # Replace 'Your Legend Label' with an appropriate label for your legend
# Save the plot in 300 DPI
plt.savefig('iso.png', dpi=300)
# Show the plot
plt.show()